In [27]:
%run Markovify.py

In [29]:
example.swym_y.value_counts()

 1.0      9024
 3.0      1589
 104.0     904
 6.0       557
 7.0       421
 4.0       369
-1.0       250
 8.0         1
Name: nextaction, dtype: int64

In [30]:
example.rfc_score()

/home/jjc/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.66076455033759152